## Reading data from database

In [38]:
import os
from mypackage import finder, userinput
from sqlalchemy import create_engine, MetaData, Table, select

def get_chunks_from_db():

    data_dir = os.path.join(finder.get_git_root(), "data")
    db_name = 'documents.db'
    db_path = os.path.join(data_dir, db_name)

    engine = create_engine(f"sqlite:////{db_path}")

    metadata = MetaData()

    table = Table(
        'documents',
        metadata,
        autoload_with=engine
    )

    stmt = select(table)
    connection = engine.connect()

    results = connection.execute(stmt).fetchall()
    for result in results:
        print(result)

get_chunks_from_db()

(0, 1, 'language models are few-shot learners\ntom b. brown∗ benjamin mann∗ nick ryder∗ melanie subbiah∗\njared kaplan† prafulla dhariwal arvind neelakantan  ... (210 characters truncated) ... ric sigler mateusz litwin scott gray\nbenjamin chess jack clark christopher berner\nsam mccandlish alec radford ilya sutskever dario amodei\nabstract')
(1, 1, 'abstract\nwe demonstrate that scaling up language models greatly improves task-agnostic,\nfew-shot performance, sometimes even becoming competitive w ... (188 characters truncated) ... uage model, and test its performance in the few-shot setting. for all\ntasks, gpt-3 is applied without any gradient updates or ﬁne-tuning, with tasks')
(2, 1, 'and few-shot demonstrations speciﬁed purely via text interaction with the model.\ngpt-3 achieves strong performance on many nlp datasets, including t ... (207 characters truncated) ... o training on large web corpora.\n1 introduction\nnlp has shifted from learning task-speciﬁc representations and desi

In [ ]:
def get_embeddings_from_chunks():
    metadata = [{'page_number': pn} for pn in df_chunks['page_number']]

    embeddings = OllamaEmbeddings(
        model="llama3.2"
    )



    vector_store = QdrantVectorStore(
        client=client,
        collection_name="documents",
        embedding=embeddings,
    )
    vector_store.add_texts(texts, metadatas=metadata)






client = QdrantClient(":memory:")

vector_size = len(embeddings.embed_query("test"))

if not client.collection_exists("documents"):
    client.create_collection(
        collection_name="documents",
        vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
    )

In [35]:
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_ollama import ChatOllama


# Construct a tool for retrieving context
@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs




llm = ChatOllama(
    model="llama3.2",
    temperature=0.1,
)

tools = [retrieve_context]
# If desired, specify custom instructions
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(llm, tools, system_prompt=prompt)

In [36]:
user_query = userinput.get_user_input("Stelle deine Frage!", default="Worum geht es in dem Dokument?")
for step in agent.stream(
    {"messages": [{"role": "user", "content": user_query}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Welche Informationen enthält das Dokument?
================================== Ai Message ==================================
Tool Calls:
  retrieve_context (c6048f02-67fa-495f-9f09-26a85c808bfa)
 Call ID: c6048f02-67fa-495f-9f09-26a85c808bfa
  Args:
    query: {'type': 'string'}
    object: <nil>
    key: <nil>
================================= Tool Message =================================
Name: retrieve_context

Error invoking tool 'retrieve_context' with kwargs {'query': {'type': 'string'}, 'object': '<nil>', 'key': '<nil>'} with error:
 query: Input should be a valid string
 Please fix the error and try again.
================================== Ai Message ==================================

Es tut mir leid, aber ich habe nicht genug Informationen über das Dokument, um Ihnen eine genaue Antwort zu geben. Könnten Sie bitte mehr Kontext oder Details zum Dokument bereitstellen? Ich werde mein Bestes tun, u